In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statistics import mode

In [2]:
df = pd.read_csv('data.csv')
df

,Id,Age,Income,Gender,Marital Status,Buys
0,1,<21,High,Male,Single,No
1,2,<21,High,Male,Married,No
2,3,21-35,High,Male,Single,Yes
3,4,>35,Medium,Male,Single,Yes
4,5,>35,Low,Female,Single,Yes
5,6,>35,Low,Female,Married,No
6,7,21-35,Low,Female,Married,Yes
7,8,<21,Medium,Male,Single,No
8,9,<21,Low,Female,Married,Yes
9,10,>35,Medium,Female,Single,Yes


In [3]:
del df['Id']

### One-Hot Encoding

In [4]:
for category in df['Age'].unique():
    df[category] = df['Age'].apply(lambda x: 1 if x==category else 0)

del df['Age']

In [5]:
for category in df['Income'].unique():
    df[category] = df['Income'].apply(lambda x: 1 if x==category else 0)

del df['Income']

In [6]:
for category in df['Gender'].unique():
    df[category] = df['Gender'].apply(lambda x: 1 if x==category else 0)

del df['Gender']

In [7]:
for category in df['Marital Status'].unique():
    df[category] = df['Marital Status'].apply(lambda x: 1 if x==category else 0)

del df['Marital Status']

In [8]:
df['Buys'] = df['Buys'].apply(lambda x: 1 if x=='Yes' else 0)

In [9]:
df

,Buys,<21,21-35,>35,High,Medium,Low,Male,Female,Single,Married
0,0,1,0,0,1,0,0,1,0,1,0
1,0,1,0,0,1,0,0,1,0,0,1
2,1,0,1,0,1,0,0,1,0,1,0
3,1,0,0,1,0,1,0,1,0,1,0
4,1,0,0,1,0,0,1,0,1,1,0
5,0,0,0,1,0,0,1,0,1,0,1
6,1,0,1,0,0,0,1,0,1,0,1
7,0,1,0,0,0,1,0,1,0,1,0
8,1,1,0,0,0,0,1,0,1,0,1
9,1,0,0,1,0,1,0,0,1,1,0


---

### Node Class

In [10]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value

### Tree Class

In [11]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
        self.num_features = 0
        self.features = []
        
    def fit(self, x, y, features=[]):
        ''' function to train the tree '''
        
        num_samples, num_features = np.shape(x)
        self.num_features = num_features
        
        self.features = features
        if len(features) < self.num_features:
            self.features = [f'x{i}' for i in range(self.num_features)]
        
        dataset = np.concatenate((x, y.reshape(-1,1)), axis=1)
        self.root = self.build_tree(dataset)
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree ''' 
        
        x = dataset[:, :-1]
        y = dataset[:, -1]
        
        num_samples, num_features = np.shape(x)
        
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset)
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features
        for feature_index in range(self.num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y)
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child):
        ''' function to compute information gain '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        return gain
    
    def gini_index(self, y):
        ''' function to compute gini index '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, y):
        ''' function to compute leaf node '''
        return mode(y)
#         y = list(y)
#         return max(y, key=y.count)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''

        if not tree:
            tree = self.root

        if tree.value is not None:
            print(f'target({tree.value})')

        else:
            print(self.features[tree.feature_index], "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft: " % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright: " % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def predict(self, x):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(row, self.root) for row in x]
        return preditions
    
    def make_prediction(self, row, node):
        ''' function to predict a single data point '''
        
        if node.value != None: 
            return node.value
        
        value = row[node.feature_index]
        if value <= node.threshold:
            return self.make_prediction(row, node.left)
        else:
            return self.make_prediction(row, node.right)

In [12]:
X = df.iloc[:,1:]
X

,<21,21-35,>35,High,Medium,Low,Male,Female,Single,Married
0,1,0,0,1,0,0,1,0,1,0
1,1,0,0,1,0,0,1,0,0,1
2,0,1,0,1,0,0,1,0,1,0
3,0,0,1,0,1,0,1,0,1,0
4,0,0,1,0,0,1,0,1,1,0
5,0,0,1,0,0,1,0,1,0,1
6,0,1,0,0,0,1,0,1,0,1
7,1,0,0,0,1,0,1,0,1,0
8,1,0,0,0,0,1,0,1,0,1
9,0,0,1,0,1,0,0,1,1,0


In [13]:
Y = df['Buys']
Y

0     0
1     0
2     1
3     1
4     1
5     0
6     1
7     0
8     1
9     1
10    1
11    1
12    1
13    0
Name: Buys, dtype: int64

In [14]:
x = np.array(X)
y = np.array(Y)

x_train, y_train = x[:-4], y[:-4]
x_test, y_test = x[-4:], y[-4:]

In [15]:
classifier = DecisionTreeClassifier(max_depth=3)
classifier.fit(x_train,y_train, features=X.columns.values)

In [16]:
classifier.print_tree()

<21 <= 0 ? 0.16333333333333333
 left: Single <= 0 ? 0.11111111111111102
  left: 21-35 <= 0 ? 0.5
    left: target(0)
    right: target(1)
  right: target(1)
 right: Low <= 0 ? 0.375
  left: target(0)
  right: target(1)


In [17]:
y_pred = classifier.predict(x_test)
print('y_pred:', y_pred)
print('x_test:', y_test)

y_pred: [0, 1, 1, 0]
x_test: [1 1 1 0]


In [18]:
y_pred = classifier.predict(x)

tn = 0
tp = 0
fn = 0
fp = 0
for i in range(len(df)):
    if y_pred[i] == 0 and y[i] == 0:
        tn += 1
    if y_pred[i] == 0 and y[i] == 1:
        fn += 1
    if y_pred[i] == 1 and y[i] == 0:
        fp += 1
    if y_pred[i] == 1 and y[i] == 1:
        tp += 1

print('accuracy:', (tn+tp)/(tn+fn+fp+tp))

accuracy: 0.9285714285714286


---

In [19]:
X.columns.values

array(['<21', '21-35', '>35', 'High', 'Medium', 'Low', 'Male', 'Female',
       'Single', 'Married'], dtype=object)

In [20]:
def encode(features):
    encoded = np.zeros(len(X.columns), dtype=int)
    for feature in features:
        for i in range(len(X.columns)):
            if X.columns.values[i].lower() == feature.lower():
                encoded[i] = 1
                break
    return encoded.reshape(1,-1)

In [21]:
age = '<21'
income = 'low'
gender = 'female'
status = 'married'

classifier.predict(encode([age, income, gender, status]))

[1]